In [ ]:
#| default_exp utils

# Utils

> App setup and utility functions

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.common import fast_app as ori_fast_app, FastHTML as ori_FastHTML
from fasthtml.jupyter import *
from fastlucide import *
from enum import Enum
from fastcore.meta import delegates
import re

### Basecoat headers

In [ ]:
#| export
# Additional script needed for sliders
slider_script = Script("""
const updateSlider = (el) => {
    const min = parseFloat(el.min || 0);
    const max = parseFloat(el.max || 100);
    const value = parseFloat(el.value);
    const percent = (max === min) ? 0 : ((value - min) / (max - min)) * 100;
    el.style.setProperty('--slider-value', `${percent}%`);
};
""")

In [ ]:
#| export
# For some reason the text tailwind classes are not being properly generated
text_css = Style("""
.text-muted-foreground { color: var(--muted-foreground); }
.text-foreground { color: var(--foreground); }
.hover\\:text-foreground:hover { color: var(--foreground); }
.bg-accent { background-color: var(--accent); }
.bg-border { background-color: var(--border); }
.bg-card { background-color: var(--card); }
.bg-muted { background-color: var(--muted); }
.bg-primary { background-color: var(--primary); }
.bg-background { background-color: var(--background); }
.border-border { border-color: var(--border); }
""")
theme_script = Script("""
(() => {
  const stored = localStorage.getItem('themeMode');
  const dark = stored ? stored === 'dark' : matchMedia('(prefers-color-scheme: dark)').matches;
  if (dark) document.documentElement.classList.add('dark');
  
  document.addEventListener('basecoat:theme', (e) => {
    const mode = e.detail?.mode || (document.documentElement.classList.contains('dark') ? 'light' : 'dark');
    const isDark = mode === 'dark';
    document.documentElement.classList.toggle('dark', isDark);
    localStorage.setItem('themeMode', isDark ? 'dark' : 'light');
  });
})();
""")

ss = SvgSprites('l-')
Icon = ss # Renaming for more clarity when it is used

deps = {
    'scripts': [
        'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4',
        'https://cdn.jsdelivr.net/npm/lit@3/dist/index.js',
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/all.min.js'
    ],
    'links': [
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/basecoat.cdn.min.css'
    ]
}

def make_hdrs(deps):
    scripts = tuple(Script(src=url) for url in deps['scripts'])
    links = tuple(Link(rel='stylesheet', href=url) for url in deps['links'])
    return (theme_script, ss) + scripts + links + (text_css, slider_script)

basecoat_hdrs = make_hdrs(deps)

NameError: name 'Style' is not defined

### Fasthtml

In [ ]:
#| export
@delegates(ori_FastHTML, keep=True, but=["pico"])
def FastHTML(hdrs=None, pico=False, **kwargs):
    hdrs = basecoat_hdrs + (hdrs or ())
    return ori_FastHTML(hdrs=hdrs, pico=pico, **kwargs)

In [ ]:
FastHTML

<function __main__.FastHTML(hdrs=None, pico=False, *, debug=False, routes=None, middleware=None, title: str = 'FastHTML page', exception_handlers=None, on_startup=None, on_shutdown=None, lifespan=None, ftrs=None, exts=None, before=None, after=None, surreal=True, htmx=True, default_hdrs=True, sess_cls=<class 'starlette.middleware.sessions.SessionMiddleware'>, secret_key=None, session_cookie='session_', max_age=31536000, sess_path='/', same_site='lax', sess_https_only=False, sess_domain=None, key_fname='.sesskey', body_wrap=<function noop_body>, htmlkw=None, nb_hdrs=False, canonical=True, **kwargs)>

In [ ]:
app = FastHTML(session_cookie="mysession")
rt = app.route

In [ ]:
#| export
def get_preview(app=None): 
    if not app: app = FastHTML(session_cookie="mysession")
    return partial(HTMX, app=app, host=None, port=None)
p = get_preview()

In [ ]:
c = Div(
    Button('Hey there', cls='btn-outline'),
)
p(c)

In [ ]:
#| export
def slugify(s):
    return re.sub(r"[&/\s]+", "-", s).strip("-").lower()

In [ ]:
slugify("This content")

'this-content'

In [ ]:
#| export
# To easily preview items in a larger container
def Window(*args, cls="h-96"):
    return Div(*args, cls="w-full flex flex-col items-center justify-center {cls}")

In [ ]:
#| export
def pw(*args, **kwargs):
    return p(Div(Window(*args, **kwargs), cls="h-100 w-full flex flex-col justify-center items-center"))

In [ ]:
pw(c)

In [ ]:
#| export
class VEnum(Enum):
    def __str__(self): return self.value
    def __add__(self, b): return f"{self.value} {b}"
    def __radd__(self, a): return f"{a} {self.value}"

In [ ]:
class TestEnum(VEnum):
    test1 = "testing-one"
    test2 = "testing-two"

In [ ]:
TestEnum.test1 + TestEnum.test2

'testing-one testing-two'

In [ ]:
"h2" + TestEnum.test1

'h2 testing-one'

When deplying this notebook with nbdev on github, I'm getting this error:

Successfully built fhbasecoat
Installing collected packages: fhbasecoat
  Attempting uninstall: fhbasecoat
    Found existing installation: fhbasecoat 0.0.1
    Uninstalling fhbasecoat-0.0.1:
      Successfully uninstalled fhbasecoat-0.0.1
Successfully installed fhbasecoat-0.0.1
Quarto is not installed. We will download and install it for you.
Installing or upgrading quarto -- this requires root access.
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-a7dc6c0ec882>", line 5, in <module>
    from fastlucide import *
ModuleNotFoundError: No module named 'fastlucide'
The above exception was the direct cause of the following exception:
Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/process.py", line 205, in _process_chunk
    return [fn(*args) for args in chunk]
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/process.py", line 205, in <listcomp>
    return [fn(*args) for args in chunk]
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/fastcore/parallel.py", line 63, in _call
    return g(item)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/serve_drv.py", line 22, in main
    if src.suffix=='.ipynb': exec_nb(src, dst, x)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/serve_drv.py", line 16, in exec_nb
    cb()(nb)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/processors.py", line 292, in __call__
    def __call__(self, nb): return self.nb_proc(nb).process()
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/process.py", line 129, in process
    for proc in self.procs: self._proc(proc)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/process.py", line 122, in _proc
    for cell in self.nb.cells: self._process_cell(proc, cell)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/process.py", line 113, in _process_cell
    if callable(proc) and not _is_direc(proc): cell = opt_set(cell, proc(cell))
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/processors.py", line 252, in __call__
    raise Exception(f"Error{' in notebook: '+title if title else ''} in cell {cell.idx_} :\n{cell.source}") from self.k.exc
Exception: Error in notebook: Utils in cell 3 :
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.common import fast_app as ori_fast_app, FastHTML as ori_FastHTML
from fasthtml.jupyter import *
from fastlucide import *
from fastcore.meta import delegates
"""
The above exception was the direct cause of the following exception:
Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.19/x64/bin/nbdev_docs", line 7, in <module>
    sys.exit(nbdev_docs())
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/fastcore/script.py", line 125, in _f
    return tfunc(**merge(args, args_from_prog(func, xtra)))
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/quarto.py", line 311, in nbdev_docs
    cache,cfg,path = _pre_docs(path, n_workers=n_workers, **kwargs)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/quarto.py", line 207, in _pre_docs
    cache = proc_nbs(path, n_workers=n_workers, **kwargs)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/serve.py", line 80, in proc_nbs
    parallel(nbdev.serve_drv.main, files, n_workers=n_workers, pause=0.01, **kw)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/fastcore/parallel.py", line 134, in parallel
    return L(r)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/fastcore/foundation.py", line 105, in __call__
    return super().__call__(x, *args, **kwargs)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/fastcore/foundation.py", line 113, in __init__
    items = listify(items, *rest, use_list=use_list, match=match)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/fastcore/basics.py", line 79, in listify
    elif is_iter(o): res = list(o)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/process.py", line 575, in _chain_from_iterable_of_lists
    for element in iterable:
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/_base.py", line 621, in result_iterator
    yield _result_or_cancel(fs.pop())
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/_base.py", line 319, in _result_or_cancel
    return fut.result(timeout)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/_base.py", line 458, in result
    return self.__get_result()
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._exception
Exception: Error in notebook: Utils in cell 3 :
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.common import fast_app as ori_fast_app, FastHTML as ori_FastHTML
from fasthtml.jupyter import *
from fastlucide import *
from fastcore.meta import delegates
Error: Process completed with exit code 1.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The error is clear: `ModuleNotFoundError: No module named 'fastlucide'`. The GitHub Actions environment doesn't have `fastlucide` installed, even though your local environment does.

You need to add `fastlucide` as a dependency in your project's `settings.ini` file. Look for the `requirements` line and add it there.

Does your `settings.ini` file currently list `fastlucide` in the requirements? If not, that's what needs to be fixed.